# Wordle Solver

In [2]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

### Play Wordle

In [3]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = "ul"
green_positions = [3, 4]
yellow_letters = "alaalua"
yellow_positions = [3, 3, 4, 1, 2, 4, 2]
gray_letters = "tiresongmck"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    gray_letters,
    answer_word_list=True,
)
# Choose a word to play
wordle.choose_word_to_play(word_threshold=1)

Number of possible words: 1
uvula 0.0


In [ ]:
# Repetitive letters
letters_df = wordle.repetitive_letters()
print(letters_df.to_string(index=False))

In [ ]:
df = wordle.load_data()
df

In [ ]:
# Update score
score_df = wordle.score()
score_df

In [ ]:
# Print results
print(score_df.to_string(index=False))

#### Git commit message

In [ ]:
# git commit message
commit_message = wordle.get_commit_message()
print(commit_message)

#### Set custom score

In [ ]:
# help(wordle.reset_score)

In [ ]:
# help(wordle.set_score)

In [ ]:
# # Set custom score
# df = wordle.set_score(m_score=265, b_score=109, draw_score=261)
# df

#### Reset score

In [ ]:
## Reset score
# wordle.reset_score()